<a href="https://colab.research.google.com/github/RebeccaKessler/Machine_Learning/blob/main/Codes/CamemBert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install accelerate -U
!pip install pandas numpy matplotlib
!pip install scikit-learn seaborn
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

## Import Packages

In [ ]:
from transformers import Trainer, TrainingArguments, CamembertTokenizer, CamembertForSequenceClassification, CamembertConfig
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import pipeline
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import pipeline
import optuna

## Define Functions

In [ ]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
 # Define pre-processing function
 class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = str(self.data.iloc[idx]['sentence'])
        label = int(self.data.iloc[idx]['encoded_labels'])

        encoding = self.tokenizer.encode_plus(
            sentence,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

##Hyperparameter Optimization

In [ ]:
# Load the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)

In [ ]:
# Apply LabelEncoder to labels
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

# Split the dataset into training/validation and final test set
train_val_data, test_data = train_test_split(data, test_size=0.2, stratify=data['encoded_labels'], random_state=42)

In [ ]:
#load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Define objective function for hyperoptimization with Optuna
def objective(trial):
    # Define hyperparameter search space
    learning_rate = trial.suggest_loguniform("learning_rate", 10e-5, 20e-5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 7)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        logging_dir='./logs',
        logging_steps=10,
        warmup_steps=1000,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True
    )


    # Split the training and validation sets from the training data
    train_data, val_data = train_test_split(train_val_data, test_size=0.2, stratify=train_val_data['encoded_labels'], random_state=42)

    # Tokenize datasets
    train_dataset = CustomDataset(train_data, tokenizer)
    eval_dataset = CustomDataset(val_data, tokenizer)

    # Load the pre-trained model pre-trained
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

    # Initialize and define the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_accuracy"]

# Create and optimize the Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

# Extract the best hyperparameters
best_params = study.best_trial.params

# Print the best combination of parameters
print(f"Best trial accuracy: {study.best_trial.value}")
print(f"Best parameters: {study.best_trial.params}")

# Define the training arguments using the best hyperparameters
best_training_args = TrainingArguments(
    output_dir='./best_results',
    num_train_epochs=best_params['num_train_epochs'],
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    logging_dir='./best_logs',
    logging_steps=10,
    warmup_steps=1000,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True
)

# Tokenize the full training dataset
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
train_dataset = CustomDataset(train_val_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# Load the pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Initialize the trainer with the best hyperparameters
trainer = Trainer(
    model=model,
    args=best_training_args,
    train_dataset=train_dataset,
    eval_dataset = test_dataset,
    compute_metrics=compute_metrics
)

# Train the model with the best hyperparameters
trainer.train()

# Evaluate on the test dataset
test_result = trainer.evaluate(test_dataset)

# Print the evaluation results
print(f"Test accuracy: {test_result['eval_accuracy']}")
print(f"Test loss: {test_result['eval_loss']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Best trial accuracy: 0.5481770833333334
Best parameters: {'learning_rate': 0.00019018657617818227, 'per_device_train_batch_size': 8, 'num_train_epochs': 7}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.220600,1.214878,0.475000,0.466016,0.475000,0.457842
2,1.390200,1.248128,0.436458,0.345197,0.436458,0.360764
3,1.003100,1.205222,0.514583,0.536569,0.514583,0.499778
4,0.821500,1.250303,0.517708,0.517131,0.517708,0.513728
5,0.679400,1.392480,0.547917,0.558301,0.547917,0.539413
6,0.324200,1.697584,0.551042,0.559273,0.551042,0.552212
7,0.150900,1.966278,0.566667,0.586113,0.566667,0.571075


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test accuracy: 0.5666666666666667
Test loss: 1.966277837753296


## Fine-tune the CamemBert Model (with optimized parameters)

In [ ]:
# Load the data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/training_data.csv'
data = pd.read_csv(url)

In [ ]:
# Encode data
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['difficulty'])

In [ ]:
# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Create empty lists for statistics
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []


# K-Fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(data, data['encoded_labels'])):
    train_data = data.iloc[train_idx]
    test_data = data.iloc[val_idx]

    # Tokenize datasets
    train_dataset = CustomDataset(train_data, tokenizer)
    test_dataset = CustomDataset(test_data, tokenizer)

    # Load pre-trained model
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_fold_{fold}',
        learning_rate=0.00015, #manually adjusted
        num_train_epochs=7,
        per_device_train_batch_size=16,
        warmup_steps=1000,
        weight_decay=0.02, #increase in weight_decay to control for overfitting
        logging_dir=f'./logs_fold_{fold}',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=500,
        fp16=True,
    )

    # Initialize and define trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_result = trainer.evaluate()
    accuracy_list.append(eval_result['eval_accuracy'])
    precision_list.append(eval_result['eval_precision'])
    recall_list.append(eval_result['eval_recall'])
    f1_list.append(eval_result['eval_f1'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.761500,1.732193,0.394792,0.423927,0.394792,0.361972
200,1.399400,1.346618,0.416667,0.431343,0.416667,0.344877
300,1.187000,1.182873,0.471875,0.467929,0.471875,0.464877
400,1.120400,1.120708,0.508333,0.498647,0.508333,0.477427
500,0.894000,1.260015,0.465625,0.484740,0.465625,0.457919
600,1.002400,1.157585,0.512500,0.500446,0.512500,0.487288
700,1.004400,1.251707,0.497917,0.516105,0.497917,0.484788
800,0.957700,1.640252,0.455208,0.500307,0.455208,0.437991
900,0.858600,1.413117,0.507292,0.523554,0.507292,0.502523
1000,0.853200,1.294814,0.528125,0.534899,0.528125,0.518675


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.756200,1.735944,0.325000,0.261548,0.325000,0.220452
200,1.351700,1.318539,0.486458,0.475052,0.486458,0.436722
300,1.134400,1.129538,0.562500,0.560605,0.562500,0.555625
400,1.060400,1.078498,0.542708,0.552866,0.542708,0.517471
500,1.027300,1.029820,0.583333,0.607540,0.583333,0.587413
600,0.919400,1.025854,0.578125,0.582881,0.578125,0.573433
700,1.055400,1.137699,0.507292,0.505512,0.507292,0.488782
800,0.818900,1.028609,0.581250,0.581273,0.581250,0.578540
900,0.926600,1.070752,0.585417,0.586854,0.585417,0.580220
1000,0.772800,1.162071,0.542708,0.576410,0.542708,0.537520


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.748500,1.730184,0.346875,0.491938,0.346875,0.256051
200,1.356300,1.348517,0.412500,0.425292,0.412500,0.349985
300,1.156400,1.388955,0.385417,0.429374,0.385417,0.346332
400,1.205900,1.222035,0.465625,0.487483,0.465625,0.454852
500,0.954800,1.141804,0.507292,0.496066,0.507292,0.489342
600,1.033500,1.192470,0.500000,0.498294,0.500000,0.482043
700,0.852900,1.169837,0.510417,0.518592,0.510417,0.500988
800,0.770800,1.212782,0.525000,0.542333,0.525000,0.517352
900,0.931400,1.228996,0.539583,0.530976,0.539583,0.527594
1000,0.754900,1.484882,0.510417,0.510720,0.510417,0.500325


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.759200,1.739950,0.360417,0.377560,0.360417,0.277469
200,1.407200,1.339270,0.442708,0.426927,0.442708,0.403478
300,1.224400,1.215503,0.483333,0.491042,0.483333,0.478830
400,1.099500,1.138976,0.487500,0.551207,0.487500,0.483728
500,0.909800,1.062393,0.552083,0.564707,0.552083,0.552624
600,0.928400,1.038775,0.551042,0.548342,0.551042,0.536471
700,0.919200,1.114752,0.510417,0.540060,0.510417,0.501299
800,0.968600,1.149027,0.526042,0.590328,0.526042,0.520523
900,0.848200,1.133928,0.543750,0.537257,0.543750,0.522601
1000,0.687900,1.200999,0.510417,0.580871,0.510417,0.505146


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.750500,1.732699,0.414583,0.513511,0.414583,0.363501
200,1.388400,1.315899,0.468750,0.453964,0.468750,0.437149
300,1.208800,1.159067,0.525000,0.520521,0.525000,0.513986
400,1.053500,1.098582,0.546875,0.540853,0.546875,0.540612
500,0.915100,1.049877,0.543750,0.554690,0.543750,0.531808
600,0.883700,1.073721,0.535417,0.533400,0.535417,0.530446
700,1.006600,1.241957,0.469792,0.503051,0.469792,0.458860
800,0.766300,1.171304,0.553125,0.573099,0.553125,0.551977
900,0.789900,1.125248,0.570833,0.625019,0.570833,0.571724
1000,0.629800,1.218406,0.552083,0.592893,0.552083,0.556223


In [ ]:
# Compute overall statistics of the model
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
overall_precision = sum(precision_list) / len(precision_list)
overall_recall = sum(recall_list) / len(recall_list)
overall_f1 = sum(f1_list) / len(f1_list)

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")
print(f"Overall F1 Score: {overall_f1:.4f}")

## Re-train on Full Dataset

In [ ]:
# Combine training and validation data
combined_data = pd.concat([train_data, test_data])

In [ ]:
#Load pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenize the dataset
final_dataset = CustomDataset(combined_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00015,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.02,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and define the trainer
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset,
    compute_metrics=None
)

# Retrain the model on the whole dataset
final_trainer.train()

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
10,1.799400
20,1.790700
30,1.796500
40,1.791800
50,1.788500
60,1.782600
70,1.779900
80,1.782200
90,1.770900
100,1.754400


Step,Training Loss
10,1.799400
20,1.790700
30,1.796500
40,1.791800
50,1.788500
60,1.782600
70,1.779900
80,1.782200
90,1.770900
100,1.754400


TrainOutput(global_step=2100, training_loss=0.774715739545368, metrics={'train_runtime': 365.7318, 'train_samples_per_second': 91.871, 'train_steps_per_second': 5.742, 'total_flos': 2210212240588800.0, 'train_loss': 0.774715739545368, 'epoch': 7.0})

In [ ]:
# Save the final fine-tuned model and tokenizer
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/sentencepiece.bpe.model',
 './final_model/added_tokens.json')

## Make Predictions

In [ ]:
# Load the unlabelled data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/unlabelled_test_data.csv'
unlabelled_data = pd.read_csv(url)

In [ ]:
# Load the saved fine-tuned model and tokenizer
model_path = './final_model'
model = CamembertForSequenceClassification.from_pretrained(model_path)
tokenizer = CamembertTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Create a prediction pipeline
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, framework='pt', device=device)

# Predict labels for the unlabelled data
predictions = classifier(unlabelled_data['sentence'].tolist())

# Decode the numeric labels to original labels using the loaded LabelEncoder
predicted_labels = [label_encoder.inverse_transform([int(pred['label'].split('_')[-1])])[0] for pred in predictions]

# Create a new DataFrame with predictions
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': predicted_labels
})

# Save the results to a new CSV file
results_df.to_csv('predicted_difficulties.csv', index=False)

print("Predictions saved to 'predicted_difficulties.csv'")


Predictions saved to 'predicted_difficulties.csv'


## Re-train on Extended Dataset

In [ ]:
# Load extended data set
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/combined_random_french_sentences.csv'
full_data = pd.read_csv(url)

In [ ]:
# Apply LabelEncoder to labels
label_encoder = LabelEncoder()
full_data['encoded_labels'] = label_encoder.fit_transform(full_data['difficulty'])

In [ ]:
# Load pre-trained model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Tokenize dataset
full_dataset = CustomDataset(full_data, tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=0.00015,
    num_train_epochs=7,
    per_device_train_batch_size=16,
    warmup_steps=1000,
    weight_decay=0.02,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='no',
    save_strategy="steps",
    save_steps=500,
    fp16=True,
    )

# Re-initialize and define the trainer
final_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_dataset,
    compute_metrics=None
)

# Retrain the model on the extended dataset
final_trainer.train()


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
10,1.790900
20,1.793400
30,1.792600
40,1.789500
50,1.780300
60,1.782200
70,1.760300
80,1.748100
90,1.728100
100,1.716300


In [ ]:
#save the model
model_path_full = "./fine_tuned_model_full"
model.save_pretrained(model_path_full)
tokenizer.save_pretrained(model_path_full)

## Make Predictions

In [ ]:
# Load the unlabelled data
url = 'https://raw.githubusercontent.com/RebeccaKessler/Machine_Learning/main/unlabelled_test_data.csv'
unlabelled_data = pd.read_csv(url)

In [ ]:
# Load the fine-tuned model and tokenizer for predictions
model = CamembertForSequenceClassification.from_pretrained(model_path_full)
tokenizer = CamembertTokenizer.from_pretrained(model_path_full)

# Define prediction pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Predict the labels of the unlabelled data
predictions = classifier(unlabelled_data['sentence'].tolist())
predicted_labels = [label_encoder.inverse_transform([int(pred['label'].split('_')[-1])])[0] for pred in predictions]

# Create a new dataframe with predictions
results_df = pd.DataFrame({
    'id': unlabelled_data['id'],
    'difficulty': predicted_labels
})
results_df.to_csv('predicted_difficulties_full.csv', index=False)